In [1]:
# Importing dependencies
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
from sklearn.svm import SVC
from collections import Counter

In [2]:
# Read in the data
df = pd.read_csv("../Resources/Churn_Modelling_2.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.shape

(10000, 14)

In [4]:
# Dropping unessential columns
df = df.drop(["RowNumber", "CustomerId", "Surname"], axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df.shape

(10000, 11)

In [6]:
#create new table with Age outlier removed
a_q1 = df['Age'].quantile(0.25)
a_q3 = df['Age'].quantile(0.75)
a_iqr = a_q3-a_q1
a_l_out = a_q1-1.5*a_iqr
a_u_out = a_q3+1.5*a_iqr
a_out_df = df[(df['Age']<a_u_out) & (df['Age']>a_l_out)]
a_out_df.shape

(9589, 11)

In [7]:
a_out_df.to_csv('cleaned_a_out_not_encoded.csv')

In [8]:
# Split preprocessed data into features and target
X = a_out_df.copy()
X = X.drop("Exited", axis=1)
X = pd.get_dummies(X)

# Create our target
y = a_out_df["Exited"]

In [9]:
# Check the balance of our target values
y.value_counts()

0    7646
1    1943
Name: Exited, dtype: int64

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)
Counter(y_train)

Counter({0: 5746, 1: 1445})

In [11]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({0: 3787, 1: 4825})

In [13]:
model = SVC(kernel="linear")
model.fit(X_resampled, y_resampled)

SVC(kernel='linear')

In [14]:
y_pred = model.predict(X_test_scaled)

In [15]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, y_pred)
acc_score

0.72142887338829

In [16]:
# Displaying results
print("Confusion Matrix")
display(confusion_matrix(y_test, y_pred))
print(f"Accuracy Scorem  : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Confusion Matrix


array([[1284,  616],
       [ 116,  382]], dtype=int64)

Accuracy Scorem  : 0.72142887338829
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.92      0.68      0.77      0.78      0.72      0.51      1900
          1       0.38      0.77      0.68      0.51      0.72      0.52       498

avg / total       0.81      0.69      0.75      0.72      0.72      0.52      2398

